# Обработка данных о потреблении электроэнергии в домашних условиях

## Подключение библиотек

In [2]:
import pandas as pd
import numpy as np

## Данные

### Считываем данные с файла

In [3]:
file_path = './household_power_consumption.txt'
df_original = pd.read_csv(file_path, sep=';', low_memory=False)

df = df_original.copy()
df

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0
...,...,...,...,...,...,...,...,...,...
2075254,26/11/2010,20:58:00,0.946,0.000,240.430,4.000,0.000,0.000,0.0
2075255,26/11/2010,20:59:00,0.944,0.000,240.000,4.000,0.000,0.000,0.0
2075256,26/11/2010,21:00:00,0.938,0.000,239.820,3.800,0.000,0.000,0.0
2075257,26/11/2010,21:01:00,0.934,0.000,239.700,3.800,0.000,0.000,0.0


In [4]:
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], 
                                format='%d/%m/%Y %H:%M:%S')

df.set_index('datetime', inplace=True)
df.drop(columns=['Date', 'Time'], inplace=True)

df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek  # 0=Пн, 6=Вс
df['is_weekend'] = (df.index.weekday >= 5).astype(int)
df['month'] = df.index.month

df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

In [5]:
missingPercent = df.isnull().mean() * 100
nonMisssingPercent = (1 - df.isnull().mean()) * 100

missingStats = pd.DataFrame({
    "Total": df.isnull().sum(), 
    "Percent Missing": missingPercent.round(2),  
    "Percent Non-Missing": nonMisssingPercent.round(2) 
})

print(missingStats)

                       Total  Percent Missing  Percent Non-Missing
Global_active_power        0             0.00               100.00
Global_reactive_power      0             0.00               100.00
Voltage                    0             0.00               100.00
Global_intensity           0             0.00               100.00
Sub_metering_1             0             0.00               100.00
Sub_metering_2             0             0.00               100.00
Sub_metering_3         25979             1.25                98.75
hour                       0             0.00               100.00
day_of_week                0             0.00               100.00
is_weekend                 0             0.00               100.00
month                      0             0.00               100.00
hour_sin                   0             0.00               100.00
hour_cos                   0             0.00               100.00


## Заменяем пропущенные значение с помощью линейной интерполяции

In [6]:
df.replace('?', np.nan, inplace=True)
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
means = df.mean()
df.fillna(means, inplace=True)

### Анализируем кол-во пропущенных данных

In [7]:
missingPercent = df.isnull().mean() * 100
nonMisssingPercent = (1 - df.isnull().mean()) * 100

missingStats = pd.DataFrame({
    "Total": df.isnull().sum(), 
    "Percent Missing": missingPercent.round(2),  
    "Percent Non-Missing": nonMisssingPercent.round(2) 
})

print(missingStats)

                       Total  Percent Missing  Percent Non-Missing
Global_active_power        0              0.0                100.0
Global_reactive_power      0              0.0                100.0
Voltage                    0              0.0                100.0
Global_intensity           0              0.0                100.0
Sub_metering_1             0              0.0                100.0
Sub_metering_2             0              0.0                100.0
Sub_metering_3             0              0.0                100.0
hour                       0              0.0                100.0
day_of_week                0              0.0                100.0
is_weekend                 0              0.0                100.0
month                      0              0.0                100.0
hour_sin                   0              0.0                100.0
hour_cos                   0              0.0                100.0


### Подсчет уникальных значений для каждого признака, чтобы определить категории признаков

In [8]:
unique_counts = df.nunique()
unique_counts_table = pd.DataFrame({
    'Feature': unique_counts.index,
    'Unique Values': unique_counts.values
})

unique_counts_table['Percentage'] = (unique_counts_table['Unique Values'] / len(df)) * 100
print(unique_counts_table)

                  Feature  Unique Values  Percentage
0     Global_active_power           4187    0.201758
1   Global_reactive_power            533    0.025684
2                 Voltage           2838    0.136754
3        Global_intensity            222    0.010697
4          Sub_metering_1             89    0.004289
5          Sub_metering_2             82    0.003951
6          Sub_metering_3             33    0.001590
7                    hour             24    0.001156
8             day_of_week              7    0.000337
9              is_weekend              2    0.000096
10                  month             12    0.000578
11               hour_sin             21    0.001012
12               hour_cos             22    0.001060


In [9]:
for col in df.columns:
    df[col] = df[col].astype('float64')

print(df.dtypes)

Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
hour                     float64
day_of_week              float64
is_weekend               float64
month                    float64
hour_sin                 float64
hour_cos                 float64
dtype: object


In [10]:
target = 'Global_active_power'
correlations = df.corr(numeric_only=True)[target].abs().sort_values(ascending=False)
print(correlations)

Global_active_power      1.000000
Global_intensity         0.998889
Sub_metering_3           0.638555
Sub_metering_1           0.484401
Sub_metering_2           0.434569
Voltage                  0.399762
hour                     0.279953
Global_reactive_power    0.247017
hour_sin                 0.186999
hour_cos                 0.109704
is_weekend               0.083841
day_of_week              0.065320
month                    0.032520
Name: Global_active_power, dtype: float64


In [11]:
high_corr_features = correlations[correlations >= 0.2].index.tolist()

df_filtered = df[high_corr_features]

print("Оставшиеся признаки:")
print(high_corr_features)

Оставшиеся признаки:
['Global_active_power', 'Global_intensity', 'Sub_metering_3', 'Sub_metering_1', 'Sub_metering_2', 'Voltage', 'hour', 'Global_reactive_power']


In [13]:
df_filtered.to_csv('preprocessed_data.csv', index=True)